In [1]:
import json
import os
import sys
import boto3

Set the variables

In [2]:

region = 'us-east-1'
input_pdf_file = "./data/letter.pdf" 
chunk_size = 1000
chunk_overlapp = 100
bedrock_max_token = 4096
#reduce_chain_max_token = 1000 # 3000 #default loadsummarychain(token_max = reduce_chain_max_token))


Get a handle to the AWS Bedrock service

In [3]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name=region)

Read the PDF from the local file system

In [4]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(input_pdf_file)
pages = loader.load()

print(len(pages))

10


Use a text splitter to create the chunks

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=chunk_size, chunk_overlap=chunk_overlapp
)
docs = text_splitter.split_documents(pages)
print(len(docs))

40


Create a handle to the Bedrock LLM

In [6]:
from langchain.llms.bedrock import Bedrock
modelId = "amazon.titan-tg1-large"

llm_model = Bedrock(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": bedrock_max_token,
        "stopSequences": [],
        "temperature": 0,
        "topP": 1,
    },
    client=bedrock_runtime, #boto3_bedrock,
)

If Ollama is installed the below can be used

In [7]:
from langchain.llms import Ollama

# get the LLM
#llm_model = Ollama(base_url="http://localhost:11434",
 #            #model="wizard-vicuna:latest",
             #model="nous-hermes:13b",
   #          model="llama2:13b",
            #model="codellama:13b",
  #           verbose=True,
   #          temperature=0.0)

The map reduce prompt and the summarization prompt

In [8]:
from langchain import PromptTemplate
map_prompt = """
Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

combine_prompt = """
Write a concise summary of the following text delimited by triple backquotes that includes the following elements:
* A title that accurately reflects the content of the text.
* An introduction paragraph that provides an overview of the topic.
* Approximately twenty bullet points that list the key points of the text.
* A conclusion paragraph that summarizes the main points of the text.
```{text}```
BULLET POINT SUMMARY:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

load_map_reduce_chain(
    llm: BaseLanguageModel,
    map_prompt: BasePromptTemplate = map_reduce_prompt.PROMPT,
    combine_prompt: BasePromptTemplate = map_reduce_prompt.PROMPT,
    combine_document_variable_name: str = "text",
    map_reduce_document_variable_name: str = "text",
    collapse_prompt: Optional[BasePromptTemplate] = None,
    reduce_llm: Optional[BaseLanguageModel] = None,
    collapse_llm: Optional[BaseLanguageModel] = None,
    verbose: Optional[bool] = None,
    token_max: int = 3000,
    callbacks: Callbacks = None,
    *,
    collapse_max_retries: Optional[int] = None,
    **kwargs: Any,

Create the map reduce summarization chain that will summarize each document on it’s own in a “map” step and then “reduce” the summaries into a final summary

In [11]:
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm_model, 
                                    chain_type="map_reduce", 
                                    verbose=False,  
                                    map_prompt=map_prompt_template, 
                                    combine_prompt=combine_prompt_template) 
                                #, token_max = reduce_chain_max_token )
                                    

Run and print the summarization

In [12]:
%%time
print(summary_chain.invoke(docs))

/Users/robbroehl/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (3155 > 1024). Running this sequence through the model will result in indexing errors


{'input_documents': [Document(page_content='As I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and \nenergized by what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic \nyears in recent memory, and with some of our own operating challenge s to boot, we still found a \nway to grow demand (on top of the unprecedented growth we experienced in the first half of the \npandemic). We innovated in our largest businesses to meaningfully improve customer experience \nshort and long term. And, we made impor tant adjustments in our investment decisions and the way \nin which we’ll invent moving forward, while still preserving the long -term investments that we \nbelieve can change the future of Amazon for customers, shareholders, and employees.  \n \nWhile there were an unusual number of simultaneous challenges this past year, the reality is that if \nyou operate in large, dynamic, global market segments with many capable and well -

Title: Amazon Web Services (AWS) Investment in Large Language Models (LLMs) for Improved Customer Experiences and Generative AI Technology

Introduction:
Amazon Web Services (AWS) is investing heavily in large language models (LLMs) to enhance customer experiences, democratize access to generative AI technology for companies of all sizes, and offer various LLMs with AWS security and privacy features.

Bullet Point Summary:

1. AWS is investing in LLMs to improve customer experiences.
2. The company aims to democratize access to generative AI technology for businesses of all sizes.
3. AWS offers various LLMs with security and privacy features.
4. The company's focus on LLMs is driven by the potential for significant impact on customers, shareholders, and Amazon in the future.
5. AWS has a strong consumer business of $434B in 2022, but the majority of the market still resides in physical stores.
6. Only 10% of global IT spending has migrated to the cloud, leaving significant room for growth as more businesses transition to the cloud.
7. Amazon is optimistic about its future growth potential due to its focus on customer experiences, innovation, and hard work.
8. The company has ongoing projects and experiences that are still in their early stages.
9. AWS is confident that its best days are ahead.
10. LLMs have the potential to revolutionize developer productivity with applications like CodeWhisperer, which generates code suggestions in real-time.
11. AWS is committed to delivering innovative solutions that meet customer needs and drive growth for the company.
12. The company's investment in LLMs reflects its commitment to staying ahead of the curve in terms of technology and customer expectations.
13. AWS aims to provide customers with access to cutting-edge AI technology that is secure, reliable, and easy to use.
14. The company's focus on generative AI technology has the potential to transform industries and create new opportunities for businesses of all sizes.
15. AWS is well-positioned to capitalize on the growing demand for cloud computing and AI technology due to its strong brand, extensive customer base, and commitment to innovation.
CPU times: user 2.26 s, sys: 1.2 s, total: 3.46 s

Bedrock
Token indices sequence length is longer than the specified maximum sequence length for this model (2129 > 1024). Running this sequence through the model will result in indexing errors

Jeff Bezos, CEO of Amazon, discusses the company's decision to continue investing in AWS (Amazon Web Services) during a challenging period in 2008-2009. AWS has since become a significant part of Amazon's business, generating annual revenue of $85 billion and transforming how customers manage their technology infrastructure. Bezos emphasizes the importance of embracing change and adapting to new circumstances. Amazon has undergone a comprehensive review of its initiatives, shutting down some businesses and reevaluating others to ensure they have the potential to drive revenue, operating income, free cash flow, and return on invested capital. Despite the success of working from home during the pandemic, Bezos believes that collaborating and inventing is easier and more effective when teams are working together in person.

Amazon has made significant changes to its fulfillment network to improve fulfillment costs and speed of delivery. During the pandemic, the consumer business grew at an extraordinary clip, doubling the fulfillment center footprint and accelerating building a last-mile transportation network. This has resulted in steady productivity gains and cost reductions over the last few quarters. Additionally, Amazon has made larger structural changes to set itself up better to deliver lower costs and faster speed for many years to come. These changes include reevaluating how its US fulfillment network was organized and leveraging its larger fulfillment center footprint to move from a national fulfillment network to a regionalized network model.

AWS has an $85B annualized revenue run rate and is still early in its adoption curve, but faces short-term headwinds due to companies being more cautious in spending. However, AWS takes a different tack, focusing on building customer relationships and a business that outlasts all of us. Customers are not cost-cutting as much as cost-optimizing to take their resources and apply them to emerging and inventive new customer experiences. AWS's new customer pipeline is robust, and many companies are opting out of managing their infrastructure and preferring to move to AWS for agility, innovation, cost-efficiency, and security benefits. Chip development is a good example, with Graviton2-based compute instances delivering up to 40% better price-performance and Graviton3 chips providing 25% better performance than the Graviton2 processors. AWS has also invested in lower-cost GPUs for machine learning training and inference.

Amazon Web Services (AWS) has made significant strides in the machine learning and inference markets. In 2022, they launched their first training chip, Trainium, which is up to 140% faster than GPU-based instances at up to 70% lower cost. They also launched their first inference chips, Inferentia, in 2019, which have saved companies like Amazon over a hundred million dollars in capital expense. Their Inferentia2 chip, which just launched, offers up to four times higher throughput and ten times lower latency than their first Inferentia processor.

AWS is still in the early stages of its evolution and has a chance for unusual growth in the next decade. Amazon's Advertising business is uniquely effective for brands, as it can tailor sponsored products to be relevant to what customers are searching for. This leads to advertising that is more useful for customers and performs better for brands. Amazon Marketing Cloud (AMC) is a "clean room" (secure digital environment) in which advertisers can run custom audience and campaign analytics across a range of first and third-party inputs to generate advertising and business insights.

AWS and the Advertising team have collaborated to enable companies to store their data in AWS, operate securely in AMC with Amazon and other third-party data sources, perform analytics in AWS, and activate advertising on Amazon or third-party publishers through the Amazon Demand-Side Platform. They see future opportunity to thoughtfully integrate advertising into their video, live sports, audio, and grocery products.

Amazon has built a sustainable, long-lasting, growing company by inventing and working on long-term customer experiences. They have expanded from selling Books to adding categories like Music, Video, Electronics, and Toys, and their international Stores expansion drove $118B of revenue in 2022. They have invested in new international geographies, including India, Brazil, Mexico, Australia, various European countries, the Middle East, and parts of Africa, and have worked with partners to deliver solutions for customers. They believe that this investment in serving a broader geographical footprint will allow them to help more customers across the world and build a larger free cash flow-generating consumer business.

Amazon has been in the grocery business for over 20 years, offering more than three million items and larger pack sizes. To serve more customers' grocery needs, they need a broader physical store footprint. Whole Foods Market is a large and growing business, and Amazon Fresh is the brand they've been experimenting with. Amazon Business is an investment that leverages ecommerce and logistics capabilities to pursue a large market segment. It has thrived by delivering selection, value, and convenience in a business procurement setting. Amazon Business launched in 2015 and drives $35B in annualized gross sales, with over six million active customers, including 96 of the global Fortune 100 companies.

Buy with Prime was invented to help merchants drive conversion from views to purchases. It allows third-party brands and sellers to offer their products on their own websites to Amazon Prime members, offering fast, free Prime shipping and seamless checkout. It has recently been made available to all US merchants, increasing shopper conversion on third-party websites.

Amazon has expanded its business by offering shopping sites, pursuing large retail market segments, and using unique assets to help merchants sell more effectively. In 2003, AWS was a classic example of this, while Amazon Healthcare and Kuiper are potential analogues. Amazon Pharmacy is a full-service, online pharmacy that offers transparent pricing, easy refills, and savings for Prime members. However, customers have expressed a strong desire for Amazon to provide a better alternative to the broader healthcare experience. In July 2022, Amazon announced its acquisition of One Medical, a patient-focused experience with a digital app and offices in cities across the US. Kuiper is another example of Amazon innovating for customers in an area with high customer need, creating a low-Earth orbit satellite system to deliver high-quality broadband internet service to places around the world.

Amazon's Jeff Bezos discussed the company's future plans during his annual letter to shareholders. He highlighted the importance of AWS, Amazon's cloud computing platform, as a key driver of growth and innovation. Bezos also discussed the company's investment in new technologies, such as artificial intelligence (AI), machine learning, and the Internet of Things (IoT).

One of the key areas of investment for Amazon is in AI and machine learning. The company has been developing its own AI technology, including large language models (LLMs), for several years. LLMs are trained on large datasets and can generate human-like text and images. Amazon believes that LLMs will transform various industries, such as healthcare, finance, and customer service.

The company is also investing in machine learning chips, which are designed to accelerate the processing of AI tasks. These chips are being used in Amazon's AWS cloud infrastructure and are also being sold to other companies.

Another area of investment for Amazon is in the IoT. The company has been developing a range of IoT products, such as smart speakers, security cameras, and home automation systems. These products are designed to connect people's homes and devices to the internet and to enable them to control and monitor their environment using voice commands or mobile apps.

Bezos also discussed the company's efforts to reduce its carbon footprint and promote sustainability. The company has set a goal to be carbon neutral by 2040 and has been investing in renewable energy projects and reducing the use of plastic in its packaging.

Overall, Bezos' letter highlighted the company's commitment to innovation and growth and its investment in new technologies that will shape the future of industries.